In [2]:
import os
import json
import rasterio
import numpy as np
from matplotlib import pyplot as plt
import csv
import rasterio.mask

In [ ]:
import os
path = 'dmspols'

imagefiles = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if 'web.stable_lights.avg_vis.tif' in file:
            imagefiles.append(os.path.join(r, file))
        


In [30]:
#makes list a of staes in json file to give as input in raster_array_state function
states=[]
data_states=json.load(open("states.json"))
for x in data_states["features"]:
    states.append(x['id'])


In [38]:
#raster array function for states, to get luminosity of a state
# we open the source image as a raster object, to get the reqired state we crop raster by calling rasterio.mask.mask function 
# documentation of rasterio.mask.mask:' https://rasterio.readthedocs.io/en/latest/api/rasterio.mask.html#rasterio.mask.raster_geometry_mask'
# the function has parameters first is raster object second is geometry of region of interest here it is the state's geometry, which is extracted from json file of India's states ,
#the outputs of mask function have a tuple which has intensity of the state, we sum all elements to get total intensity  



def raster_array_state(image,state):
    sumi=0
    for x in data_states['features']:
        if (x['id']==state):
            state_geo=[x['geometry']]
    
    with rasterio.open(image) as src:
        out_imagest,out_transform = rasterio.mask.mask(src,state_geo , all_touched=True, nodata=0.0,crop=True)   
    raster_array=  out_imagest[0]
    for row in range (len(raster_array)):
            for col in range(len(raster_array[0])):
                if(raster_array[row][col]<0.0):
                    raster_array[row][col]=0.0
                sumi = sumi + raster_array[row][col]
    return sumi


In [ ]:
#to find luminosity for all states in given image
#we call raster_array_state function with image and the required state as input(which is taken fron list of states,so we get data of all states from one image) )
#output of funtion with name of state and month is written in csv file
for img in imagefiles:
    with open('datastates.csv','w', newline='') as f:
        thewriter=csv.writer(f)
        for i in states:
            thewriter.writerow([imf,i,raster_array_state(img,i)])

In [59]:
#makes list a of cities in json file to give as input in raster_array_state function

data_cities=json.load(open('cities.json'))
data_cities=data_cities['features']
cities=[]
for x in data_cities:
        y= x['properties']
        cities.append(y['NAME_2'])
    

In [14]:
#raster array function for districts, to get luminosity of a state
# we open the source image as a raster object, to get the reqired state we crop raster by calling rasterio.mask.mask function 
# documentation of rasterio.mask.mask:' https://rasterio.readthedocs.io/en/latest/api/rasterio.mask.html#rasterio.mask.raster_geometry_mask'
# the function has parameters first is raster object second is geometry of region of interest here it is the district's geometry, which is extracted from json file of India's districts ,
#the outputs of mask function have a tuple which has intensity of the district, we sum all elements to get total intensity  


def raster_array_district(image,city):
    sumi=0
    for x in data_cities:
        y= x['properties']
        if(y['NAME_2']==city):
            geo=[x['geometry']]
            
            
    with rasterio.open(image) as src:
        out_imagecity,out_transform = rasterio.mask.mask(src,geo , all_touched=True, nodata=0.0,crop=True)   
    raster_array=  out_imagecity[0]
    for row in range (len(raster_array)):
            for col in range(len(raster_array[0])):
                if(raster_array[row][col]<0.0):
                    raster_array[row][col]=0.0
                sumi = sumi + raster_array[row][col]
    return sumi


In [62]:
#to find luminosity for all districts in given image
#we call raster_array_state function with image and the required district as input(which is taken fron list of states,so we get data of all districts from one image)
#output of funtion with name of district and month is written in csv file
for img in imagefiles:
    with open('datacities.csv','w', newline='') as f:
            thewriter=csv.writer(f)
            for j in cities:
                thewriter.writerow([img,j,raster_array_district(img,j)])
